#30DayMapChallenge - Day 17: A New Tool
Tutorial Source: https://opengeoai.org/examples/train_car_detection/

**1. Setup**

In [1]:
%pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.9/632.9 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import geoai

**2. Download Sample Data**

In [3]:
train_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/cars_7cm.tif"
)
train_vector_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/car_detection.geojson"
test_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/cars_test_7cm.tif"
)

In [4]:
train_raster_path = geoai.download_file(train_raster_url)
train_vector_path = geoai.download_file(train_vector_url)
test_raster_path = geoai.download_file(test_raster_url)

**3. Visualise Sample Data**

In [5]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_url)

In [6]:
geoai.view_raster(test_raster_url)

**4. Create Training Data**

In [7]:
out_folder = "output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=512,
    stride=256,
    buffer_radius=0,
)


Raster info for cars_7cm.tif:
  CRS: EPSG:3857
  Dimensions: 8351 x 4463
  Resolution: (0.07464844311342499, 0.07464346097829833)
  Bands: 3
  Bounds: BoundingBox(left=-10622651.27699905, bottom=3462200.907507864, right=-10622027.88785061, top=3462534.04127421)
Loaded 963 features from car_detection.geojson
Vector CRS: EPSG:4326
Reprojecting features from EPSG:4326 to EPSG:3857


Generated: 544, With features: 462: 100%|██████████| 544/544 [00:32<00:00, 16.60it/s]


------- Export Summary -------
Total tiles exported: 544
Tiles with features: 462 (84.9%)
Average feature pixels per tile: 23885.1
Output saved to: output

------- Georeference Verification -------


**5. Train Object Detection Model**

In [8]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=3,
    pretrained=True,
    batch_size=4,
    num_epochs=100,
    learning_rate=0.005,
    val_split=0.2,
)

Using device: cuda
Found 544 image files and 544 label files
Training on 435 images, validating on 109 images
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


100%|██████████| 170M/170M [00:00<00:00, 196MB/s]


Epoch: 1, Batch: 1/109, Loss: 2.2783, Time: 5.10s
Epoch: 1, Batch: 11/109, Loss: 1.3406, Time: 11.65s
Epoch: 1, Batch: 21/109, Loss: 0.7524, Time: 11.62s
Epoch: 1, Batch: 31/109, Loss: 0.4468, Time: 11.99s
Epoch: 1, Batch: 41/109, Loss: 0.4018, Time: 12.15s
Epoch: 1, Batch: 51/109, Loss: 0.5231, Time: 12.09s
Epoch: 1, Batch: 61/109, Loss: 0.2520, Time: 11.65s
Epoch: 1, Batch: 71/109, Loss: 0.4885, Time: 12.25s
Epoch: 1, Batch: 81/109, Loss: 0.4133, Time: 11.95s
Epoch: 1, Batch: 91/109, Loss: 0.4633, Time: 12.11s
Epoch: 1, Batch: 101/109, Loss: 0.3559, Time: 11.64s
Epoch 1/100: Train Loss: 0.6235, Val Loss: inf, Val IoU: 0.6821
Saving best model with IoU: 0.6821
Epoch: 2, Batch: 1/109, Loss: 0.5536, Time: 3.08s
Epoch: 2, Batch: 11/109, Loss: 0.2749, Time: 12.13s
Epoch: 2, Batch: 21/109, Loss: 0.5179, Time: 11.90s
Epoch: 2, Batch: 31/109, Loss: 0.3953, Time: 11.68s
Epoch: 2, Batch: 41/109, Loss: 0.3669, Time: 11.99s
Epoch: 2, Batch: 51/109, Loss: 0.4466, Time: 12.45s
Epoch: 2, Batch: 61/

**6. Run Inference**

In [9]:
masks_path = "cars_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

In [10]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=3,
)

Processing 270 windows with size 512x512 and overlap 256...


308it [00:30,  9.97it/s]

Inference completed in 31.01 seconds
Saved prediction to cars_prediction.tif


**7. Vectorise Mask**

In [11]:
output_path = "cars_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

Processing 445 features...


Converting features: 100%|██████████| 445/445 [00:02<00:00, 150.92shape/s]


Saving to cars_prediction.geojson...
Done!


**8. Visualise Results**

In [12]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

In [13]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)

Map(center=[-95.4219, 29.673650000000002], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…